https://deap.readthedocs.io/en/master/overview.html

Create Types

In [1]:
# create types

from deap import base, creator
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

initalization of individual and population

In [2]:
# initialization

import random
from deap import tools

# 遺伝子型のサイズ
IND_SIZE = 10

toolbox = base.Toolbox()
toolbox.register("attribute", random.random) # 1遺伝子
toolbox.register("individual", tools.initRepeat, creator.Individual,
                toolbox.attribute, n=IND_SIZE) # 1個体はIND_SIZE個のattributeからなる
toolbox.register("population", tools.initRepeat, list, toolbox.individual) # 1集団はindividualのリスト型

operators: mate, mutate, select, evaluate

In [7]:
# 評価関数
def evaluate(individual):
    return sum(individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

algorithms

In [4]:
# 集団
pop = toolbox.population(50)
print(type(pop), len(pop), type(pop[0]), len(pop[0]))
print(pop)

<class 'list'> 50 <class 'deap.creator.Individual'> 10
[[0.0038305340330141435, 0.10428090650031308, 0.9874416041238755, 0.48094808340290274, 0.3435258706176326, 0.9580017163596877, 0.9077218672756753, 0.33031688617716437, 0.5430043126671746, 0.15744044891833553], [0.3871565464377378, 0.3944285598082242, 0.43134383068847215, 0.04890313841905747, 0.2837171438194135, 0.8890371679926178, 0.8988775230367012, 0.39004142614521786, 0.40470195296750267, 0.5078425747142254], [0.15254388686992615, 0.17210599891583356, 0.6854607979258304, 0.5006456023231401, 0.984524569957128, 0.8170696183442715, 0.4298613514218058, 0.6002425914415862, 0.3458991848389502, 0.35354162696880653], [0.5412207756733214, 0.5808484952233021, 0.8466403798357818, 0.5983257138874546, 0.557152876396953, 0.43241982640439824, 0.5505347701866026, 0.2409149323757357, 0.7337952799317691, 0.45115146727048494], [0.03634966342679846, 0.29764610095223276, 0.20799192184023052, 0.6429215667317916, 0.3952695110726525, 0.2933633920797678

In [19]:
# 交差率、突然変異率、世代数
CXPB, MUTPB, NGEN = 0.5, 0.2, 40

# 全個体の評価値を計算
fitnesses = map(toolbox.evaluate, pop)
#print(list(fitnesses))

# 計算した評価値を個体の属性に設定。
# 1個体づつにevaluteしないのは、map使ったほうが速いからか？
# EAは評価計算が時間かかるから妥当ではある
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit # なぜvaluesと複数形なのか？
    print(ind, ind.fitness.values)

[-19.76154806359707, -18.826070897408826, -33.93890463425446, -19.43136749139236, -15.674051415978088, -19.67984786458592, -13.878723520588785, -25.82251060755247, -13.690009732749546, -22.9163217192577] (-203.61935594736522,)
[-19.55493531013093, -18.826070897408826, -33.93890463425446, -19.754245355664533, -14.128859269997545, -21.080009896290314, -14.601049987697904, -26.62575695393885, -13.690009732749546, -22.9163217192577] (-205.11616375739064,)
[-19.55493531013093, -18.826070897408826, -33.93890463425446, -19.754245355664533, -15.674051415978088, -19.67984786458592, -13.878723520588785, -26.62575695393885, -13.690009732749546, -22.9163217192577] (-204.53886740455766,)
[-19.55493531013093, -18.826070897408826, -33.93890463425446, -19.754245355664533, -15.674051415978088, -19.67984786458592, -13.878723520588785, -26.62575695393885, -13.690009732749546, -22.9163217192577] (-204.53886740455766,)
[-19.76154806359707, -18.826070897408826, -33.93890463425446, -19.43136749139236, -15.67

In [24]:
# 世代繰り返し、交叉、突然変異等の仕組みは全部こちらで作るのか。
# pyevolveだと決め打ちで楽だったな
# もう何世代か進化させたい！となってもやりづらいな。

# 世代を+1000したが速いな。一瞬で終わった。
# だが適応度が上がらない。突然変異が弱いな。
for g in range(NGEN+ 1000): # 世代数繰り返し
    # toolbox.register("select", tools.selTournament, tournsize=3)で登録したトーナメント選択
    offspring = toolbox.select(pop, len(pop))
    # print("offspring", type(offspring), len(offspring))
    #  clone() method that duplicates any element it is passed as argument
    # 次世代の個体集団を生成するためクローン
    offspring = list(map(toolbox.clone, offspring))

    # 偶数、奇数で分割し、隣あう個体で交叉
    for child1, child2 in zip(offspring[::2], offspring[1::2]): 
        if random.random() < CXPB:
            toolbox.mate(child1, child2) # toolbox.register("mate", tools.cxTwoPoint)
            # 親世代のfitnessが残っているから消す
            del child1.fitness.values
            del child2.fitness.values
            # あとでまたまとめてevalしている. map使用して。

    # なんだか効率悪い気がするな。乱数に対し何か演算して、個体のidxにしたほうがいいのでは
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant) # toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
            del mutant.fitness.values

    # 今回の例では、実数のリストだから異常な遺伝子型にならないが、
    # 普通突然変異・交叉で遺伝子壊れるからな。壊れていないかチェックが必要。
    # valid属性は勝手に評価されてるようだ。
    # 複雑な遺伝子型の場合は自分でvalid更新の関数定義は必要そうだ。
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # 親集団を小集団で上書きし、次世代へ。
    pop[:] = offspring
    

print(pop)

# 最良個体とかどうやって出力するんだ？
# 解析しずらいな。

[[-239.46301535243606, -222.9813915712711, -238.17255879608047, -234.71733284337495, -222.93277652327137, -224.91859698681088, -230.80529509851328, -237.4897620505137, -235.83647064680048, -223.59823766325775], [-240.52748800896143, -222.9813915712711, -238.17255879608047, -234.71733284337495, -222.93277652327137, -224.91859698681088, -230.80529509851328, -237.4897620505137, -235.83647064680048, -223.3768241236149], [-239.46301535243606, -222.9813915712711, -238.17255879608047, -234.71733284337495, -221.20119081843862, -224.91859698681088, -230.80529509851328, -237.4897620505137, -235.83647064680048, -223.3768241236149], [-240.52748800896143, -222.9813915712711, -238.17255879608047, -234.71733284337495, -222.93277652327137, -224.91859698681088, -230.80529509851328, -237.4897620505137, -235.83647064680048, -223.3768241236149], [-239.46301535243606, -222.9813915712711, -238.17255879608047, -234.71733284337495, -222.93277652327137, -224.91859698681088, -232.70890890679627, -235.5800481653

In [22]:
for ind in pop:
    print(ind, ind.fitness.values)

[-24.592316601812357, -20.5813053196399, -36.30202030717951, -21.860094545151714, -20.780263466270288, -24.71569312559384, -15.900788534847562, -32.13059929217638, -20.536549325167247, -23.757671696039967] (-241.15730221387875,)
[-24.592316601812357, -21.346982875987806, -36.27863153283415, -21.860094545151714, -20.780263466270288, -24.71569312559384, -15.900788534847562, -32.13059929217638, -20.536549325167247, -27.28050698196166] (-245.42242628180298,)
[-24.592316601812357, -21.346982875987806, -36.27863153283415, -21.860094545151714, -20.780263466270288, -24.71569312559384, -15.900788534847562, -32.13059929217638, -20.536549325167247, -25.830158503900392] (-243.97207780374174,)
[-24.592316601812357, -21.346982875987806, -36.27863153283415, -21.860094545151714, -20.780263466270288, -24.71569312559384, -15.900788534847562, -32.13059929217638, -20.536549325167247, -25.830158503900392] (-243.97207780374174,)
[-24.592316601812357, -20.5813053196399, -36.27863153283415, -21.86009454515171